# 指针

什么是指针?这个在python中可没有.

上语文课的时候,老师最长说的话是不是"请将课本翻到xxx页"?在那一页上有课上的内容,我们要看的是内容,但却使用页码标记内容,这就是指针的核心思想了.

**指针**是变量的地址,与之对应的是内存空间的标记,一本有页码的笔记本,在某一页上可以写着日记,也可以画着画,它只是被标记的空间而已,内容取决于需要.

### 变量的三要素

一个变量必然有三样内容:

+ 地址,内存空间的标记
+ 值,存放的具体内容
+ 变量名,在程序中用于标记变量的符号

在多数语言中还要加上一个类型,用以让编译器或者解释器知道该如何处理这个变量

### 取地址符&和指针运算符\*

+ 要取到变量的地址,只用在变量前使用`&`就可以得到了
+ 如果我们想通过变量的地址获取它的值,那只要在地址前使用\*就可以了

## 指针变量

存储变量地址的变量就是指针变量,他的声明形式很简单

```c
类型* 变量名
```
这样变量中就是存的地址了,而要取出该地址中的值,同样的只要使用指针运算符即可

## 指针运算

指针也就是地址,在C++中表现为可以运算的正整数数值,因此,它和int,long之类一样,是可以运算的,所用的运算符也是通常的运算符.

## 指针与数组

数组变量实际上就是数组第一个元素的指针,因此要取第一个元素的值也可以使用指针运算符,其后面的元素也只要在数组变量的基础上加上长度后再做指针运算即可,不过最安全最直观的方式还是用下标

## 函数与指针

C++语言中的精髓之一就在于函数与指针互动了

### 指针作为参数

指针作为参数就可以理解为所谓的传引用,以地址作为参数,函数的操作会影响引用的内容,这样几百年函数的内存被回收,操作的结果也会生效

最典型的,交换两个变量的值

In [1]:
%%writefile src/C8/exchange.cpp
#include <stdio.h>

void exchange(int* ptr_a,int* ptr_b){
    int temp;
    temp = *ptr_a;
    *ptr_a = *ptr_b;
    *ptr_b = temp;
    return;
}

int main(void) {
    int a = 78,b = 28;
    printf("before exchange a=%d,b=%d\n",a,b);
    exchange(&a,&b);
    printf("after exchange a=%d,b=%d\n",a,b);
    return 0;
}


Overwriting src/C8/exchange.cpp


In [2]:
!g++-7 -o bin/exchange src/C8/exchange.cpp

In [3]:
!bin/exchange

before exchange a=78,b=28
after exchange a=28,b=78


#### 指针参数的保护

我们如果传递的是指针,那么就意味着内容是可变的,于是就又被篡改的危险,但如果我们希望不被篡改,那么最好的办法就是将这个指针引用的内容在函数中保护起来,这就用到了`const`参数了,这个被称作常量指针

In [4]:
%%writefile src/C8/safe_ptr.cpp
#include <stdio.h>

int main(void) {
    const int a = 78,b = 28;
    int c = 18;
    const int* p = &a;
    printf("p:%d\n",*p);
    //*p = 58//报错,*p无法重新复制
    p = &b;//可以给p重新复制,
    printf("p:%d\n",*p);
    return 0;
}

Overwriting src/C8/safe_ptr.cpp


In [5]:
!g++-7 -o bin/safe_ptr src/C8/safe_ptr.cpp

In [6]:
!bin/safe_ptr

p:78
p:28



### 指针作为返回值(指针函数)

如果要一个函数返回指针,那么就像下面这么定义

```c
int *f(int，int);
```
指针作为返回值就意味着可以返回复杂的数据结构以及数组了

### 函数指针

函数指针是指指向函数的指针,事实上函数一样也是一种储存在内存中的数据,如果要做一定的抽象,函数指针是必不可少的.定义一个函数指针如下:

```c
int (*f)(int,int);
```

形式上 指针函数和函数指针很像,也就是一对括号的不同而已,但差异非常大,指针函数是定义的函数,而函数指针是定义的指针


### 函数常量指针

C++中事实上并没有所谓常量函数,但我们确实可以指定指向函数的指针,或者函数的返回值指针,各种各样的指针为常数


```c
int (*const func)(int) = some_func;//指向函数的常量指针,通常会接一个初始化

void const *(*func)(int);//指定返回值为常量指针

void const *(*const func)(int) = func;//指向函数的常量指针,并指定返回值为常量指针
```

## 模拟高阶函数

对于函数式编程而言,高阶函数是一切的基础,学过python的一定不会陌生,C++中如何实现高阶函数呢

### 函数作为参数

求a到b间整数的和

```c
int sum_int(int a,int b){
    return a>b ?0:a + sum_int((a+1),b);
}
```

求a到b间的整数的立方和:

```c
int cube(int x){
    return (int)pow(x,3);
}

int sum_func(int a,int b,int (*func)(int)){

    return a>b ?0:(*func)(a) + sum_func((a+1),b,func);
}
```
求:
$$ \frac {1}{1 \cdot 3} + \frac {1}{5 \cdot 7} + \frac {1}{9 \cdot 11} +\cdots$$

它会非常缓慢地收敛到$$\frac {\pi}{8}$$

```c
float sum_func_stp(int a,int b,float (*const func)(int) , int (*const step)(int)){
    return a>b ?0:(*func)(a) + sum_func_stp((*step)(a),b,func,step);
}
float func(int x){
    return 1.0/(x*(x+2));
}
int stp(int x){
    return x+4;
}
float sum_pi(int a,int b){
    float (*const ptr_func)(int) = func;
    int (*const ptr_stp)(int) = stp;
    return sum_func_stp(a,b,ptr_func,ptr_stp);
}
```

完整的高阶函数例子:

In [7]:
%%writefile src/C8/high_order_func.cpp

#include <stdio.h>
#include <math.h>

int sum_int(int a,int b){
    return a>b ?0:a + sum_int((a+1),b);
}

int cube(int x){
    return (int)pow(x,3);
}
int sum_func(int a,int b,int (*const func)(int)){
    return a>b ?0:(*func)(a) + sum_func((a+1),b,func);
}

int sum_cube(int a,int b){
    int (*const ptr_func)(int) = cube;
    return sum_func(a,b,ptr_func);
}
float sum_func_stp(int a,int b,float (*const func)(int) , int (*const step)(int)){
    return a>b ?0:(*func)(a) + sum_func_stp((*step)(a),b,func,step);
}
float func(int x){
    return 1.0/(x*(x+2));
}
int stp(int x){
    return x+4;
}
float sum_pi(int a,int b){
    float (*const ptr_func)(int) = func;
    int (*const ptr_stp)(int) = stp;
    return sum_func_stp(a,b,ptr_func,ptr_stp);
}


int main(void) {
    int top = 5,bottom = 1;
    printf("range(%d,%d) = %d\n", bottom,top,sum_int(bottom,top));
    printf("range(%d,%d)的立方和为%d\n", bottom,top,sum_cube(bottom,top));
    printf("sum_pi(%d,%d)为%f\n", bottom,top,sum_pi(bottom,top));
}

Overwriting src/C8/high_order_func.cpp


In [8]:
!g++-7 -o bin/high_order_func src/C8/high_order_func.cpp

In [9]:
!bin/high_order_func

range(1,5) = 15
range(1,5)的立方和为225
sum_pi(1,5)为0.361905


## 闭包

标准C++中是不允许在函数体内定义函数的,不过g++和clang都有相应扩展,这边只讲标准C++,因此也就不多介绍了,有兴趣的可以自己百度.

## 空指针

C++中可以用`NULL`为任意指针变量赋值,他的意义是指针为空,或者说,指针变量没有指向任何地址

## nullptr[C++11]

nullptr是为了解决原来C++中NULL的二义性问题而引进的一种新的类型,因为NULL实际上代表的是0.

In [10]:
%%writefile src/C8/nullptr_test.cpp
#include <iostream>
using std::cout;
using std::endl;
        
void F(int a){  
    cout<<a<<endl;  
}  
 
void F(int *p){  
    assert(p != NULL);  
 
    cout<< p <<endl;  
}  
 
int main(){  
 
    int *p = nullptr;  
    int *q = NULL;  
    bool equal = ( p == q ); // equal的值为true，说明p和q都是空指针  
    int a = nullptr; // 编译失败，nullptr不能转型为int  
    F(0); // 在C++98中编译失败，有二义性；在C++11中调用F（int）  
    F(nullptr);  
 
    return 0;  
} 

Overwriting src/C8/nullptr_test.cpp


## void指针

void指针很特殊,它有这样的性质:
+ void指针可以指向任何数据类型,如果要将void指针赋给其他类型指针，则需要强制类型转换如：`pint= (int *)pvoid`
+ **可以用任何类型的指针对void指针进行赋值和初始化**
+ void指针无法进行指针操作
    `void* p`就是一个典型的void指针,一般void指针用在内存操作,
    
因此void指针常用在:

+ 内存操作
+ 定义函数时,如果函数的参数可以是任意类型指针，那么应声明其参数为`void*`
+ 作为指向函数的指针


# 引用(reference)

引用就是某一变量(目标)的一个别名,对引用的操作与对变量直接操作完全一样.

引用的声明方法:

```C++
类型标识符 &引用名=目标变量名；
```
如下,定义引用ra,它是变量a的引用,即别名:

```C++
int a;

int &ra=a;
```

1. &在此不是求地址运算符,而是起标识作用.

2. 类型标识符是指目标变量的类型.

3. 声明引用时,必须同时对其进行初始化.

4. 引用声明完毕后,相当于目标变量有两个名称即该目标原名称和引用名,且不能再把该引用名作为其他变量名的别名.

5. 声明一个引用,不是新定义了一个变量,它只表示该引用名是目标变量名的一个别名,它本身不是一种数据类型,因此引用本身不占存储单元,系统也不给引用分配存储单元.故:对引用求地址,就是对目标变量求地址.&ra与&a相等.

6. 不能建立数组的引用.因为数组是一个由若干个元素所组成的集合,所以无法建立一个数组的别名。


In [11]:
%%writefile src/C8/reference_test.cpp
#include<iostream>
using std::cout;
using std::endl;
        
int main(){
    int a=5;
    int &b=a;
    b=6;
    cout<<"a="<<a<<",b="<<b<<endl;//a=6,b=6
    int c=7;
    b=c;
    cout<<"a="<<a<<",b="<<b<<endl;//a=7,b=7
    return 0;
}

Overwriting src/C8/reference_test.cpp


In [12]:
!g++-7 -o bin/reference_test src/C8/reference_test.cpp

In [13]:
!./bin/reference_test

a=6,b=6
a=7,b=7
